<a href="https://colab.research.google.com/github/Rango-Zhang-Hang/NTU-Visual-Recognition-via-Vision-Language-Model-Transfer/blob/main/MasterProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#load google drive
from google.colab import drive
drive.mount('/content/drive')

import os
#%cd drive/MyDrive
os.chdir('/content/drive/MyDrive')
!ls
#load under ~/MyDrive

#delete folders DO NOT DELETE through google drive, it will only delete the directory, but leave orphan files!
#os.chdir('/content/drive/MyDrive/data/lyft/')
#!ls
#%rm -rf train_maps
#!mv "v1.01-train/v1.01-train.zip" "v1.01-train.zip"


!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py38_23.5.2-0-Linux-x86_64.sh
!ls
!chmod +x Miniconda3-py38_23.5.2-0-Linux-x86_64.sh
%env PYTHONPATH=

!./Miniconda3-py38_23.5.2-0-Linux-x86_64.sh -b -f -p /usr/local

!which conda
!conda --version
!which python
!python -V
import sys
_ = (sys.path.append("/usr/local/lib/python3.8/site-packages"))
sys.path
!conda env list

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# creat conda env
!conda create --name hh python=3.8 -y
!source activate hh

#install packages
!source activate hh && pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!source activate hh && pip install -U openmim
!source activate hh && mim install mmengine
!source activate hh && mim install 'mmcv>=2.0.0rc4'
!source activate hh && mim install 'mmdet>=3.0.0'

!conda env list

#install mmdet3d
os.chdir('/content/drive/MyDrive')
!ls
!git clone https://github.com/open-mmlab/mmdetection3d.git -b dev-1.x
# "-b dev-1.x" means checkout to the `dev-1.x` branch.
%cd mmdetection3d
!source activate hh && pip install -v -e .
# "-v" means verbose, or more output
# "-e" means installing a project in edtiable mode,
# thus any local modifications made to the code will take effect without reinstallation.

#install other packages
os.chdir('/content/drive/MyDrive/Fast-BEV')
!ls
!source activate hh && conda install -c conda-forge pretty_errors -y
!source activate hh && pip install -r requirements.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
config	Fast-BEV    MasterProject			      mmdetection3d
data	fastbev.py  Miniconda3-py38_23.5.2-0-Linux-x86_64.sh  Untitled0.ipynb
--2023-08-05 08:15:11--  https://repo.anaconda.com/miniconda/Miniconda3-py38_23.5.2-0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

config	Fast-BEV    MasterProject			      mmdetection3d
data	fastbev.py  Miniconda3-py38_23.5.2-0-Linux-x86_64.sh  Untitled0.ipynb
env: PYTHONPATH=
PREFIX=/usr/local
Unpacking payload ...
                                                                                           
Installing base environment...



Pre

The structure should be like:
```
|MyDrive
|-config
|-data
  |-lyft
|Fast-BEV
  |-mmdet3d-dlc
```

Besides, remember to actiavte conda_env 'hh' for every block using it.

In [6]:
import os
import sys
#load google drive
from google.colab import drive
drive.mount('/content/drive')
#activate conda env
!source activate hh && conda env list

#prepare the annotation files for lyft: original->fastbev type
'''
#original lyft
os.chdir('/content/drive/MyDrive/mmdetection3d')
!ls
!python tools/create_data.py lyft --root-path ../data/lyft --out-dir ../data/lyft --extra-tag lyft --version v1.01
!python tools/dataset_converters/lyft_data_fixer.py --version v1.01 --root-folder ../data/lyft
'''
'''
#fastbev lyft
import sys
sys.path.append('/content/drive/MyDrive/Fast-BEV')
print(sys.path)
os.chdir('/content/drive/MyDrive/Fast-BEV')
!ls
!source activate hh && python tools/custom_create_data.py lyft
'''

#idk why there always some images missing althrough it is there
#check the images & lidar
import os
os.chdir('/content')
!ls
from pathlib import Path
a=False
b=False
my_images_file = Path("images.zip")
my_images_dir = Path("images")
if not my_images_file.is_file():
  !cp drive/MyDrive/data/lyft/v1.01-train/images.zip images.zip
  a=True
if not my_images_dir.is_dir():
  !unzip -o images.zip
  b=True
if a and b:
  !mv -R images drive/MyDrive/data/lyft/v1.01-train/images

c=False
d=False
my_lidar_file = Path("lidar.zip")
my_lidar_dir = Path("lidar")
if not my_lidar_file.is_file():
  !cp drive/MyDrive/data/lyft/v1.01-train/lidar.zip lidar.zip
  c=True
if not my_lidar_dir.is_dir():
  !unzip -o lidar.zip
  d=True
if c and d:
  !mv -R lidar drive/MyDrive/data/lyft/v1.01-train/lidar



#modify mAP_evaluation.py & replace
print("=======================================================================")
print("Moving to Fast-BEV")
os.chdir('/content/drive/MyDrive/Fast-BEV')
!ls
!cp -rf mAP_evaluation.py /usr/local/envs/hh/lib/python3.8/site-packages/lyft_dataset_sdk/eval/detection/mAP_evaluation.py
!cp -rf lyft_metric.py /content/drive/MyDrive/mmdetection3d/mmdet3d/evaluation/metrics/lyft_metric.py
!cp -rf lyft_eval.py /content/drive/MyDrive/mmdetection3d/mmdet3d/evaluation/functional/lyft_eval.py
!cp -rf base_3d_dense_head.py /content/drive/MyDrive/mmdetection3d/mmdet3d/models/dense_heads/base_3d_dense_head.py
!cp -rf lyft_dataset.py /content/drive/MyDrive/mmdetection3d/mmdet3d/datasets/lyft_dataset.py
!cp -rf anchor_3d_generator.py /content/drive/MyDrive/mmdetection3d/mmdet3d/models/task_modules/anchor/anchor_3d_generator.py

!cp -rf custom_valloop.py /content/drive/MyDrive/Fast-BEV/mmdet3d_dlc/core/runner/custom_valloop.py
!cp -rf lyft.py /content/drive/MyDrive/Fast-BEV/configs/config_lyft.py
!cp -rf My_lyft_dataset.py /content/drive/MyDrive/Fast-BEV/mmdet3d_dlc/dataset/lyft_dataset.py
#test

sys.path.append('/content/drive/MyDrive/Fast-BEV')
os.chdir('/content/drive/MyDrive/Fast-BEV')
!rm -r work_dirs
print("loading testing...\n")
!source activate hh && python -u tools/'custom_test.py' configs/config_lyft.py weights/nuscenes.pth --launcher="none"

from google.colab import files
files.download('work_dirs/video/video_gt.mp4')
files.download('work_dirs/video/video_pred.mp4')

Streaming output truncated to the last 5000 lines.
       [-6.4001472e+01, -1.3146349e+01, -2.0158448e+00,  4.5660000e+00,
         1.8640000e+00,  1.4809999e+00, -7.5913996e-01],
       [ 7.8179364e+00,  3.5511810e-01, -8.1887627e-01,  5.1539998e+00,
         1.8040000e+00,  1.5590000e+00, -3.1145184e+00],
       [-1.7927103e+01,  3.4680616e-02, -1.2579719e+00,  3.8929999e+00,
         1.9160000e+00,  1.8020000e+00, -3.1324205e+00],
       [ 9.0520874e+01,  7.9832735e+00,  8.6836803e-01,  9.4300002e-01,
         8.4299999e-01,  1.7520000e+00, -3.0815089e+00],
       [-3.0607599e+01,  6.4221315e-02, -1.3355055e+00,  5.1479998e+00,
         2.0599999e+00,  2.0480001e+00,  3.1208529e+00],
       [ 7.1858516e+00,  1.7125265e+01, -9.3142748e-01,  4.7930002e+00,
         1.9640000e+00,  1.6480000e+00, -8.3407217e-01],
       [-1.9810404e+01,  3.4612372e+00, -1.3372303e+00,  4.5520000e+00,
         1.7680000e+00,  1.5150000e+00,  3.1337602e+00],
       [-7.5149612e+01, -5.2419834e+00, -2.223

FileNotFoundError: ignored